# utils 

> utility module for data_io, numerics, etc.
> for example, generating a pandas DataFrame for testing purpose

In [ ]:
#| default_exp utils 

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import pandas as pd
from datetime import datetime
from functools import reduce


In [ ]:
#| export
from eos.data_io.config import drivers_by_id, trucks_by_id

## Generate state

In [ ]:
#| export
def generate_state()->pd.DataFrame:
    """
    generate a pandas DataFrame for testing purpose
    """
    ts = pd.to_datetime(datetime.now())
    ts_ind1 = ts + pd.to_timedelta(np.arange(0, 4 * 20, 20), "ms")
    s = np.arange(12)
    a_1 = s[:4]
    a_2 = s[4:8]
    a_3 = s[8:]
    df_ss = pd.DataFrame(
        {"timestep": ts_ind1, "velocity": a_1, "thrust": a_2, "brake": a_3}
    )  # .set_index('timestep')
    df_ss.columns.name = "qtuple"
    ui_t = df_ss.loc[:, ["velocity", "thrust"]]
    state = df_ss.stack().swaplevel(0, 1)
    state.name = "state"
    state.index.names = ["rows", "idx"]
    state.sort_index(inplace=True)
    return state

In [ ]:
#| hide
generate_state()

## Generate action

In [ ]:
#| export
def generate_action()->pd.DataFrame:
    """
    generate a pandas DataFrame for testing purpose
    """

    s = np.arange(12)
    ts = pd.to_datetime(datetime.now())
    a = len(s) + np.arange(15)
    speed_ser = pd.Series(np.linspace(40, 60, 3), name="speed")
    row_array = a.reshape(3, 5).transpose()
    rows_df = pd.DataFrame(row_array)
    rows_df.columns = [f"r{i}" for i in np.arange(3)]
    ts_ind0 = ts + pd.to_timedelta(np.arange(5 * 20, 8 * 20, 20), "ms")
    ts_ser = pd.Series(ts_ind0, name="timestep")
    throttle_ser = pd.Series(np.linspace(0, 1.0, 5), name="throttle")
    # throttle_ser
    dfs = [rows_df, ts_ser, speed_ser, throttle_ser]
    action = (
        reduce(
            lambda left, right: pd.merge(
                left, right, how="outer", left_index=True, right_index=True
            ),
            dfs,
        )
        .stack()
        .swaplevel(0, 1)
        .sort_index()
    )
    
    action.name = "action"
    action.index.names = ["rows", "idx"]
    return action

In [ ]:
#| hide
generate_action()

## Generate reward

In [ ]:
#| export

def generate_reward()->pd.DataFrame:
    """
    generate a pandas DataFrame for testing purpose
    """
    ts = pd.to_datetime(datetime.now())
    ts_ind1 = ts + pd.to_timedelta(np.arange(0, 4 * 20, 20), "ms")
    s = np.arange(12)
    a = len(s) + np.arange(15)
    reward = (
        pd.DataFrame({"work": len(s) + len(a), "timestep": ts_ind1[0]}, index=[0])
        .stack()
        .swaplevel(0, 1)
        .sort_index()
    )
    # reward_index = (reward.name,  ts_ind[0], 0)
    reward.index.names = ["rows", "idx"]
    reward.name = "reward"
    return reward


In [ ]:
#| hide
generate_reward()

## Generate nstate

In [ ]:
#| export
def generate_nstate()->pd.DataFrame:
    """
    generate a pandas DataFrame for testing purpose
    """
    ts = pd.to_datetime(datetime.now())
    s = np.arange(12)
    a = len(s) + np.arange(15)
    reward = generate_reward()
    
    ts_ind = ts + pd.to_timedelta(5, "s") + pd.to_timedelta(np.arange(0, 4 * 20, 20), "ms")
    s = (
            np.arange(12) + len(s) + len(a) + len(reward) - 1
    )  # exclude the timestamp in reward
    a1 = s[:4]
    a2 = s[4:8]
    a3 = s[8:]
    nstate = (
        pd.DataFrame({"timestep": ts_ind, "velocity": a1, "thrust": a2, "brake": a3})
        # .set_index('timestamp')
        .stack()
        .swaplevel(0, 1)
        .sort_index()
    )
    nstate.name = "nstate"
    nstate.index.names = ["rows", "idx"]
    return nstate

In [ ]:
#| hide
generate_nstate()

## Generate observation

In [ ]:
#| export
def generate_observation()-> list[pd.Series]:
    """
    generate a list of pandas Series for testing purpose
    """

    ts = pd.to_datetime(datetime.now())
    state = generate_state()
    action = generate_action()
    reward = generate_reward()
    nstate = generate_nstate()
    
    timestamp = pd.Series([ts], name="timestamp")
    timestamp.index = pd.MultiIndex.from_product(
        [timestamp.index, [0]], names=["rows", "idx"]
    )
    timestamp_index = (timestamp.name, "", 0)
    state_index = [(state.name, *i) for i in state.index]
    reward_index = [(reward.name, *i) for i in reward.index]
    action_index = [(action.name, *i) for i in action.index]
    nstate_index = [(nstate.name, *i) for i in nstate.index]
    
    multiindex = pd.MultiIndex.from_tuples(
        [timestamp_index, *state_index, *action_index, *reward_index, *nstate_index]
    )
    observation_list = [timestamp, state, action, reward, nstate]
    observation = pd.concat(observation_list)
    observation.index = multiindex
    observation0 = observation.copy()
    observation0.loc["timestamp", "", 0] = ts + pd.Timedelta(1, "h")
    observation1 = observation.copy()
    observation1.loc["timestamp", "", 0] = ts + pd.Timedelta(2, "h")
    observation2 = observation.copy()
    observation2.loc["timestamp", "", 0] = ts + pd.Timedelta(3, "h")
    observation3 = observation.copy()
    observation3.loc["timestamp", "", 0] = ts + pd.Timedelta(4, "h")
    observation4 = observation.copy()
    observation4.loc["timestamp", "", 0] = ts + pd.Timedelta(5, "h")
    observation_list = [
        observation0,
        observation1,
        observation2,
        observation3,
        observation4,
    ]
    return observation_list


In [ ]:
#| hide
generate_observation()

## Generate MultiIndex DataFrame

In [ ]:
#| export
def generate_df_multiindex()->pd.DataFrame:

    observation_list = generate_observation()
    dfs_epi = pd.concat(observation_list, axis=1).transpose()
    dfs_epi.columns.names = ["qtuple", "rows", "idx"]
    # dfs_epi
    
    dfs_episode = dfs_epi.copy()
    dfs_episode.set_index(("timestamp", "", 0), inplace=True)
    dfs_episode.sort_index(axis=1, inplace=True)
    dfs_episode.index.name = "timestamp"
    idx = pd.IndexSlice
    state_cols_float = [("state", col) for col in ["brake", "thrust", "velocity"]]
    action_cols_float = [("action", col) for col in ["r0", "r1", "r2", "speed", "throttle"]]
    reward_cols_float = [("reward", "work")]
    nstate_cols_float = [("nstate", col) for col in ["brake", "thrust", "velocity"]]
    for col in action_cols_float + state_cols_float + reward_cols_float + nstate_cols_float:
        dfs_episode[col[0], col[1]] = dfs_episode[col[0], col[1]].astype(
            "float"
        )  # float16 not allowed in parquet
    return dfs_episode


In [ ]:
#| hide
generate_df_multiindex()

## prepend two levels of index "vehicle" and "driver" to the DataFrame object

In [ ]:
#|export
def generate_eos_df()->pd.DataFrame:
    """
    generate a pandas DataFrame for testing purpose
    """
    dfs_episode = generate_df_multiindex()
    
    ts = pd.to_datetime(datetime.now())
    dfs_episode = pd.concat(
        [dfs_episode], keys=[drivers_by_id["wang-cheng"].pid], names=["driver"]
    )
    dfs_episode = pd.concat(
        [dfs_episode], keys=[trucks_by_id["VB7"].vid], names=["vehicle"]
    )
    episodestart = ts - pd.Timedelta(1, "h")
    dfs_episode = pd.concat([dfs_episode], keys=[ts], names=["episodestart"])
    dfs_episode = dfs_episode.swaplevel(1, 0, axis=0)
    dfs_episode = dfs_episode.swaplevel(1, 2, axis=0)
    dfs_episode.sort_index(inplace=True)
    return dfs_episode


In [ ]:
#|hide
generate_eos_df()

In [ ]:
assert isinstance(generate_eos_df().index, pd.MultiIndex), f"dfs_episode.index is not a MultiIndex"

In [ ]:
from fastcore.test import *

In [ ]:
test_eq(isinstance(generate_eos_df().index, pd.MultiIndex), True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()